In [1]:
import re
import numpy as np
from nltk.tokenize import TweetTokenizer

import torch
from torchtext import data
from torch.autograd import Variable

from torchtext import data

This notebook is not about learning math. Its to take a quick look at the class that stores the hyperparameters, the class that samples the training data and the functions that generate the appropriate masks. Except for the Masks, these have veru little to do with transformers, but I place them here for transparency, which might help for debugging.

In [4]:
teaching = False # True

Options is just a place to store all your constants, hyper parameters and directory paths

In [8]:
class Options:
    def __init__(self, batchsize=4, device=-1, epochs=20, lr=0.01, 
                 beam_width=2, max_len=20, save_path='saved/weights/model_weights'):
        self.batchsize = batchsize
        self.device = device
        self.epochs = epochs
        self.lr = lr
        self.k = beam_width
        self.max_len = max_len
        self.save_path = save_path

In [9]:
if teaching:
    
    opt = Options(batchsize=2, device=torch.device("cpu"), epochs=25, 
                  lr=0.01, beam_width=3, max_len = 25, save_path = '../saved/weights/MoveDataExample')

In [10]:
class MyIterator(data.Iterator):
    '''
    patch on Torchtext's batching process that makes it more efficient
    http://nlp.seas.harvard.edu/2018/04/03/attention.html#position-wise-feed-forward-networks
    '''
    def create_batches(self):
        if self.train:
            def pool(d, random_shuffler):
                for p in data.batch(d, self.batch_size * 100):
                    p_batch = data.batch(
                        sorted(p, key=self.sort_key),
                        self.batch_size, self.batch_size_fn)
                    for b in random_shuffler(list(p_batch)):
                        yield b
            self.batches = pool(self.data(), self.random_shuffler)
            
        else:
            self.batches = []
            for b in data.batch(self.data(), self.batch_size,
                                          self.batch_size_fn):
                self.batches.append(sorted(b, key=self.sort_key))
                
def batch_size_fn(new, count, sofar):
    "Keep augmenting batch and calculate total number of tokens + padding."
    global max_src_in_batch, max_tgt_in_batch
    if count == 1:
        max_src_in_batch = 0
        max_tgt_in_batch = 0
    max_src_in_batch = max(max_src_in_batch,  len(new.src))
    max_tgt_in_batch = max(max_tgt_in_batch,  len(new.trg) + 2)
    src_elements = count * max_src_in_batch
    tgt_elements = count * max_tgt_in_batch
    return max(src_elements, tgt_elements)

class Tokenizer(object):
    
    def __init__(self):

        self.tweettokenizer = TweetTokenizer()
            
    def tokenize(self, sentence):
        sentence = re.sub(r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’\;]", " ", str(sentence))
        sentence = re.sub(r"[ ]+", " ", sentence)
        sentence = re.sub(r"\!+", "!", sentence)
        sentence = re.sub(r"\,+", ",", sentence)
        sentence = re.sub(r"\?+", "?", sentence)
        sentence = sentence.lower()
        sentence = self.tweettokenizer.tokenize(sentence)
        return sentence
    
def num_batches(train):

    for i, b in enumerate(train):
        pass
    
    return i + 1

In [11]:
def json2datatools(path = None, tokenizer = None, opt = None):

    if opt == None:
        opt = Options()
        opt.batchsize = 4
        opt.device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

    if path == None:
        path = 'saved/pairs.json' 

    if tokenizer == None:
        tokenizer = Tokenizer()
        
    input_field = data.Field(lower=True, tokenize=tokenizer.tokenize)
    output_field = data.Field(lower=True, tokenize=tokenizer.tokenize, 
                              unk_token='<unk>', init_token='<sos>', eos_token='<eos>')

    fields={'listen':('listen', input_field),'reply':('reply', output_field)} 

    trainingset = data.TabularDataset(path, format='json', fields=fields) 

    input_field.build_vocab(trainingset)
    output_field.build_vocab(trainingset)
    training_iterator = MyIterator(trainingset, batch_size=opt.batchsize, 
                        device=opt.device, repeat=False, 
                        sort_key=lambda x: (len(x.listen), len(x.reply)), 
                        train=True, shuffle=True)
    opt.src_pad = input_field.vocab.stoi['<pad>']
    opt.trg_pad = output_field.vocab.stoi['<pad>']
    return training_iterator, input_field, output_field, opt

In [12]:
if teaching:
    data_iter, infield, outfield, opt = json2datatools(path='../saved/examplepairs.json', opt=opt)

In [16]:
print(infield.vocab.stoi)
print(" ------------------------------------------------------ ")
print(outfield.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x1185944c0>>, {'<unk>': 0, '<pad>': 1, 'chloe': 2, 'you': 3, '?': 4, 'hi': 5, 'i': 6, 'are': 7, 'haha': 8, 'lol': 9, 'ok': 10, 'bye': 11, 'hello': 12, 'how': 13, 'later': 14, 'sure': 15, 'who': 16, '!': 17, 'a': 18, 'alive': 19, 'am': 20, 'any': 21, 'dont': 22, 'dunno': 23, 'go': 24, 'gotta': 25, 'ill': 26, 'joke': 27, 'know': 28, 'me': 29, 'more': 30, 'see': 31, 'talk': 32, 'tell': 33, 'to': 34, 'true': 35, 'ttyl': 36, 'yes': 37})
 ------------------------------------------------------ 
defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x118594a60>>, {'<unk>': 0, '<pad>': 1, '<sos>': 2, '<eos>': 3, '?': 4, 'you': 5, 'i': 6, 'joke': 7, '!': 8, ',': 9, 'a': 10, 'can': 11, 'cats': 12, 'do': 13, 'french': 14, 'hi': 15, 'how': 16, 'say': 17, 'tell': 18, 'thank': 19, 'thanks': 20, 'at': 21, 'bye': 22, 'for': 23, 'laughing': 24, 'my': 25, 'ttyl': 26, 'am': 27, 'beaucou

In [49]:
for i, batch in enumerate(data_iter): 
    src = batch.listen.transpose(0,1) # get an input ready
    trg = batch.reply.transpose(0,1)  # get a target ready
    break

In [50]:
src

tensor([[31,  3, 14,  2],
        [10, 36,  2, 17]])

In [51]:
trg

tensor([[ 2, 22, 26,  3],
        [ 2, 22, 26,  3]])

In [35]:
def nopeak_mask(size, opt):
    "Mask out subsequent positions. aka subsequent_mask"
    np_mask = np.triu(np.ones((1, size, size)), k=1).astype('uint8')
    np_mask =  torch.from_numpy(np_mask) == 0
    if opt.device == torch.device("cuda:0"):
      np_mask = np_mask.cuda()
    return np_mask

In [36]:
def create_masks(src, trg, opt):
    src_mask = (src != opt.src_pad).unsqueeze(-2)
    if trg is not None:
        trg_mask = (trg != opt.trg_pad).unsqueeze(-2)
        size = trg.size(1) # get seq_len for matrix
        np_mask = nopeak_mask(size, opt)
        if trg.is_cuda:
            np_mask.cuda()
        trg_mask = trg_mask & np_mask
    else:
        trg_mask = None
    return src_mask, trg_mask

In [52]:
if teaching:
    src_mask, trg_mask = create_masks(src, trg, opt)

In [53]:
src_mask

tensor([[[True, True, True, True]],

        [[True, True, True, True]]])

In [54]:
trg_mask

tensor([[[ True, False, False, False],
         [ True,  True, False, False],
         [ True,  True,  True, False],
         [ True,  True,  True,  True]],

        [[ True, False, False, False],
         [ True,  True, False, False],
         [ True,  True,  True, False],
         [ True,  True,  True,  True]]])

In [ ]:
def load_subset_weights(whole_model, opt):
    '''
    This function allows you to load saved weights from a saved model that is a subset of your model
    It looks for the named parameters that match and loads those but will not crash trying to load
    parameters that dont have a matching name
    '''
    subset_model_dict = torch.load(opt.save_path)
    whole_model_dict = whole_model.state_dict() 
    for name, param in whole_model_dict.items(): 
        if name in subset_model_dict:
            whole_model_dict[name].copy_(subset_model_dict[name])

The next place to study is notebooks/Elements.ipynb